In [1]:
%%capture
%pip install - r requirement.txt
# python==3.6.13

In [2]:
import numpy as np
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMaximize, LpMinimize, LpBinary, LpStatus, value, PULP_CBC_CMD
import pulp as pl
import time
import heapq
import pymysql
debug = 1  # debug mode
TIMEOUT = 300 # timeout 
if debug:
    pd.set_option('display.max_columns', 500)  # show all columns

### PARAMETER INPUT

`None` stands for no boundary.

Hereby its just a demo.

In [3]:
n = 100000 # number of containers

NbvCost = True  # True: max nbv; False: min cost

minTotalNbv = 10000000
maxTotalNbv = 350000000

minTotalCost = 1000000
maxTotalCost = 300000000

fleetAgeLowBound = [None, 3, None, None, None]
fleetAgeUpBound = [3, 6, None, None, None]
fleetAgeLimit = [0.2, 0.0, None, None, None]
fleetAgeGeq = [True, True, None, None, None] # True: >=, False: <=

fleetAgeAvgLimit = 5.0
fleetAgeAvgGeq = False

OnHireLimit = 0.7

weightedAgeLowBound = [3, 4, None, None, None]
weightedAgeUpBound = [6, 15, None, None, None]
weightedAgeLimit = [0.0, 0.01, None, None, None]
weightedAgeGeq = [True, True, None, None, None]

weightedAgeAvgLimit = 7.0
weightedAgeAvgGeq = False

productType = ['D4H', 'D20', 'D40', None, None]
productLimit = [0.5, 0.1, 0.0, None, None]
productGeq = [True, True, True, None, None]

lesseeType = ['MSC', 'ESSC', 'ONE', None, None]
lesseeLimit = [0.7, 0.2, 0.8, None, None]
lesseeGeq = [False, False, False, None, None]

topLesseeLimit = [0.65, 0.8, None]
topLesseeGeq = [False, False, None]

contractType = ['LT', 'LP', 'LM', None, None]
contractLimit = [0.6, 0.02, 0.0, None, None]
contractGeq = [True, True, True, None, None]

### READ DATA
TODO: read data

In [4]:
rawData = pd.read_excel(io='./20220630_Overall_Fleet_(FIR)_FCI_combined(with planned)_iflorens_simplified_new.xlsb', sheet_name='Raw', engine='pyxlsb')

(100000, 10)


In [15]:
data = rawData.iloc[n:2*n,:55]
print(data.shape)

(100000, 10)


In [16]:
data.drop('Unit Id Fz', axis=1, inplace=True)

### Ont-Hot

In [17]:
def SelectStatus(status):
    # no need to check input
    return 1 if status=='ON' else 0

data['Billing Status Fz'] = data.apply(lambda x: SelectStatus(x['Billing Status Fz']), axis=1)

In [18]:
def OneHotCoding(v, name):
    return 1 if v==name else 0

for lesseeName in data['Contract Cust Id'].value_counts().index:
    data[lesseeName] = data.apply(lambda x: OneHotCoding(x['Contract Cust Id'], lesseeName), axis=1)

for t in data['Contract Lease Type'].value_counts().index:
    data[t] = data.apply(lambda x: OneHotCoding(x['Contract Lease Type'], t), axis=1)

In [19]:
data.drop(['Contract Cust Id', 'Contract Lease Type'], axis=1, inplace=True)

In [31]:
data.sample(3)

,Cost,Product,Contract Num,Nbv,Billing Status Fz,Fleet Year Fz,Age x CEU,MSC,ONE,YANG,ESSC,MSK,COSMR,CMA,CUL,TSLINE,NBOS,HALINE,SITC,SINOKOR,INTRAPTE,HTHK,HYDAI,HAPAG,MATSN,DLIM,ATI,AMTC,CSSC,DONGJIN,CHINAV,SMLINE,VSBLLC,HARBOUR,NAMSUNG,SOGESE,SSCI,OML,SINOCL,CKLINE,DCLP,TLINE,SAMSKIP,RCL,MSCL,SYS,MUTO,XIASC,MTT,ZIM,WH,EMIRATE,GOODRICH,PANOCEAN,HTC,BOXJ,IGLO,LILM,WKS,DPL,ENGLEE,RALA,PDZ,BSLI,MALC,EQUIPE,MRTS,AMLSHP,DALCO,OBLS,ELAN,QEL,BISL,GREVILLE,CENTRANS,GOSS,DACL,APMVAD,KNOF,AEDXB3,EIMSKIP,MESSINA,PORTAU,MELL,WINSMART,WFL,COSLA,EGREN,HAS,MBF,MINIUS,FORMOSA,PANCON,LT,LP,LM,ML,FF,LF,LE
179435,14088.0,R4H,LE-ONE-05B,12352.12,1,2.23,17.840,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
151233,3655.0,D4H,LT-MSC-46,3405.77,1,1.45,2.465,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
131995,3739.0,D4H,LT-MSC-45,3406.23,1,1.77,3.009,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [33]:
grouped = data.groupby(['Product', 'Contract Num'])

### DATA PREPARATION

TODO: change the column name 

#### Container Age

Assign to `FleetAge{0}.format(i)`

In [ ]:
def SelectFleetAge(age, i):
    # check input
    if fleetAgeLowBound[i] is None:
        fleetAgeLowBound[i] = -float('inf')
    if fleetAgeUpBound[i] is None:
        fleetAgeUpBound[i] = float('inf')
    
    return 1 if fleetAgeLowBound[i]<=age<fleetAgeUpBound[i] else 0

for i in range(5):
    if fleetAgeLimit[i] is not None:
        column_name = 'FleetAge{0}'.format(i)
        print(column_name)
        data[column_name] = data.apply(lambda x: SelectFleetAge(x['Fleet Year Fz'], i), axis=1)

FleetAge0
FleetAge1


#### Weighted Age

Assign new column `WeightedAge{0}.format(i)`

In [ ]:
def SelectWeightedAge(age, i):
    # check input
    if weightedAgeLowBound[i] is None:
        weightedAgeLowBound[i] = -float('inf')
    if weightedAgeUpBound[i] is None:
        weightedAgeUpBound[i] = float('inf')
    
    return 1 if weightedAgeLowBound[i]<=age<weightedAgeUpBound[i] else 0

for i in range(5):
    if weightedAgeLimit[i] is not None:
        column_name = 'WeightedAge{0}'.format(i)
        print(column_name)
        data[column_name] = data.apply(lambda x: SelectWeightedAge(x['Age x CEU'], i), axis=1)

WeightedAge0
WeightedAge1


#### Product

Assign new column `ProductType{0}.format(i)`

In [ ]:
def SelectProductType(product, i):
    return 1 if product == productType[i] else 0

for i in range(5):
    if productLimit[i] is not None:
        column_name = 'ProductType{0}'.format(i)
        print(column_name)
        data[column_name] = data.apply(lambda x: SelectProductType(x['Product'], i), axis=1)


ProductType0
ProductType1
ProductType2


#### Lessee

Assign new column `Lessee{0}.format(i)`

In [ ]:
def SelectLessee(lessee, i):
    return 1 if lessee == lesseeType[i] else 0

for i in range(5):
    if lesseeLimit[i] is not None:
        column_name = 'Lessee{0}'.format(i)
        print(column_name)
        data[column_name] = data.apply(lambda x: SelectLessee(x['Contract Cust Id'], i), axis=1)

Lessee0
Lessee1
Lessee2


In [ ]:
# ONE HOT -- all lessee
def OneHotLessee(lessee, name):
    return 1 if lessee==name else 0

for lesseeName in data['Contract Cust Id'].value_counts().index:
    data[lesseeName] = data.apply(lambda x: OneHotLessee(x['Contract Cust Id'], lesseeName), axis=1)

#### Contract Type

Assign new column `ContractType{0}.format(i)`

In [ ]:
def SelectContractType(contract, i):
    if contract == contractType[i]:
        contract = 1
    else:
        contract = 0
    return contract

for i in range(5):
    if contractLimit[i] is not None:
        column_name = 'ContractType{0}'.format(i)
        print(column_name)
        data[column_name] = data.apply(lambda x: SelectContractType(x['Contract Lease Type'], i), axis=1)

ContractType0
ContractType1
ContractType2


### SAVE DATA

In [ ]:
if debug:
    data.to_csv('prepared_data_demo.csv')

In [ ]:
if debug:
    data = pd.read_csv('./prepared_data_demo.csv').sample(n)
data.head(3)

/home/liwuchen/anaconda3/envs/port/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (4,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unit Id Fz,Cost,Product,Contract Cust Id,Contract Lease Type,Nbv,Billing Status Fz,Fleet Year Fz,Age x CEU,FleetAge0,FleetAge1,Status,WeightedAge0,WeightedAge1,ProductType0,ProductType1,ProductType2,Lessee0,Lessee1,Lessee2,MSC,ONE,YANG,ESSC,MSK,COSMR,CMA,CUL,TSLINE,NBOS,HALINE,SITC,SINOKOR,INTRAPTE,HTHK,HYDAI,HAPAG,MATSN,DLIM,ATI,AMTC,CSSC,DONGJIN,CHINAV,SMLINE,VSBLLC,HARBOUR,NAMSUNG,SOGESE,OML,SSCI,SINOCL,CKLINE,DCLP,TLINE,SAMSKIP,RCL,MSCL,SYS,MUTO,XIASC,MTT,ZIM,WH,EMIRATE,GOODRICH,PANOCEAN,HTC,BOXJ,IGLO,WKS,LILM,DPL,PDZ,ENGLEE,RALA,EQUIPE,MALC,BSLI,DALCO,ELAN,AMLSHP,MRTS,OBLS,QEL,DACL,CENTRANS,GREVILLE,BISL,GOSS,WINSMART,EIMSKIP,APMVAD,KNOF,MESSINA,PORTAU,MELL,AEDXB3,EGREN,HAS,WFL,FORMOSA,COSLA,MBF,MINIUS,PANCON,ContractType0,ContractType1,ContractType2
58354,158354,FFAU2529423,3962.0,D4H,MSC,LT,3634.68,ON,1.65,2.805,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
52546,152546,FFAU2594809,3962.0,D4H,MSC,LT,3618.32,ON,1.71,2.907,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
62366,162366,FTAU1671338,2900.0,D20,CUL,LT,2900.00,ON,0.11,0.110,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


### Model Part

#### Prepare Parameters

convert to numpy

In [ ]:
nbv = data['Nbv'].to_numpy()
cost = data['Cost'].to_numpy()
status = data['Status'].to_numpy()

fleetAge = []
weightedAge = []
product = []
lessee = []
contract = []
for i in range(5):
    fleetAge.append(data['FleetAge{0}'.format(i)].to_numpy() if fleetAgeLimit[i] is not None else None)
    weightedAge.append(data['WeightedAge{0}'.format(i)].to_numpy() if weightedAgeLimit[i] is not None else None)
    product.append(data['ProductType{0}'.format(i)].to_numpy() if productLimit[i] is not None else None)
    lessee.append(data['Lessee{0}'.format(i)].to_numpy() if lesseeLimit[i] is not None else None)
    contract.append(data['ContractType{0}'.format(i)].to_numpy() if contractLimit[i] is not None else None)

fleetAgeAvg = data['Fleet Year Fz'].to_numpy()
weightedAgeAvg = data['Age x CEU'].to_numpy()

lesseeOneHot = {lesseeName: data[lesseeName].to_numpy() for lesseeName in data['Contract Cust Id'].value_counts().index}


if debug:
    print('nbv:', nbv.shape)
    print('cost:', cost.shape)
    print('status:', status.shape)

    print('fleetAge:', fleetAge)
    print('fleetAgeAvg:', fleetAgeAvg)
    print('weightedAge:', weightedAge)
    print('weightedAgeAvg:', weightedAgeAvg)
    print('productType:', product)
    print('lessee:', lessee)
    print('contractType:', contract)

nbv: (100000,)
cost: (100000,)
status: (100000,)
fleetAge: [array([1, 1, 1, ..., 1, 1, 1]), array([0, 0, 0, ..., 0, 0, 0]), None, None, None]
fleetAgeAvg: [1.65 1.71 0.11 ... 1.5  1.54 0.08]
weightedAge: [array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]), None, None, None]
weightedAgeAvg: [2.805 2.907 0.11  ... 2.55  2.618 0.08 ]
productType: [array([1, 1, 0, ..., 1, 1, 0]), array([0, 0, 1, ..., 0, 0, 1]), array([0, 0, 0, ..., 0, 0, 0]), None, None]
lessee: [array([1, 1, 0, ..., 1, 1, 0]), array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]), None, None]
contractType: [array([1, 1, 1, ..., 1, 1, 1]), array([0, 0, 0, ..., 0, 0, 0]), array([0, 0, 0, ..., 0, 0, 0]), None, None]


#### Constraints

TODO: confirm constraints


#### Objective

1. min Cost: `Vars @ cost`

2. max Nbv: `Vars @ nbv`

#### Define Problem

In [ ]:
def SortTop(l, n):
    topN = heapq.nlargest(n, l, key=lambda x:x[1])
    print(topN)
    return np.sum(np.stack([lesseeOneHot[topN[i][0]] for i in range(n)]), axis=0)

In [ ]:
# variables
var = np.array([LpVariable('container_{0}'.format(i), lowBound=0, cat=LpBinary) for i in range(nbv.shape[0])])
# problem
prob = LpProblem("MyProblem", LpMaximize if NbvCost else LpMinimize)

In [ ]:
# objective function 
if NbvCost:
    prob += lpSum(var * nbv)
else:
    prob += lpSum(var * cost)

/home/liwuchen/anaconda3/envs/port/lib/python3.6/site-packages/pulp/pulp.py:1704: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [ ]:
# constraints
numSelected = lpSum(var) # num of selected containers

# nbv
if maxTotalNbv is not None:
    prob += lpSum(var * nbv) <= maxTotalNbv
if minTotalNbv is not None:
    prob += lpSum(var * nbv) >= minTotalNbv
    
# cost
if maxTotalCost is not None:
    prob += lpSum(var * cost) <= maxTotalCost
if minTotalCost is not None:
    prob += lpSum(var * cost) >= minTotalCost

# status
if OnHireLimit is not None:
    prob += lpSum(var * status) >= OnHireLimit * numSelected
    
# container age
if fleetAgeAvgLimit is not None:
    if fleetAgeAvgGeq:
        prob += lpSum(var * fleetAgeAvg) >= fleetAgeAvgLimit * numSelected
    else:
        prob += lpSum(var * fleetAgeAvg) <= fleetAgeAvgLimit * numSelected
for i in range(5):
    if fleetAgeLimit[i] is not None:
        if fleetAgeGeq[i]:
            prob += lpSum(var * fleetAge[i]) >= fleetAgeLimit[i] * numSelected
        else:
            prob += lpSum(var * fleetAge[i]) <= fleetAgeLimit[i] * numSelected

# weighted age
if weightedAgeAvgLimit is not None:
    if weightedAgeAvgGeq:
        prob += lpSum(var * weightedAgeAvg) >= weightedAgeAvgLimit * numSelected
    else:
        prob += lpSum(var * weightedAgeAvg) <= weightedAgeAvgLimit * numSelected
for i in range(5):
    if weightedAgeLimit[i] is not None:
        if weightedAgeGeq[i]:
            prob += lpSum(var * weightedAge[i]) >= weightedAgeLimit[i] * numSelected
        else:
            prob += lpSum(var * weightedAge[i]) <= weightedAgeLimit[i] * numSelected

# product
for i in range(5):
    if productLimit[i] is not None:
        if productGeq[i]:
            prob += lpSum(var * product[i]) >= productLimit[i] * numSelected
        else:
            prob += lpSum(var * product[i]) <= productLimit[i] * numSelected

# # lessee
# for i in range(5):
#     if lesseeLimit[i] is not None:
#         if lesseeGeq[i]:
#             prob += lpSum(var * lessee[i]) >= lesseeLimit[i] * numSelected
#         else:
#             prob += lpSum(var * lessee[i]) <= lesseeLimit[i] * numSelected


# find top1
if topLesseeLimit[0] is not None:
    if topLesseeGeq[0]:
       prob += lpSum(var * SortTop(list({lesseeName: value(lpSum(var * lesseeOneHot[lesseeName])) for lesseeName in data['Contract Cust Id'].value_counts().index}.items()), 1)) >= topLesseeLimit[0] * numSelected
    else:
        prob += lpSum(var * SortTop(list({lesseeName: value(lpSum(var * lesseeOneHot[lesseeName])) for lesseeName in data['Contract Cust Id'].value_counts().index}.items()), 1)) <= topLesseeLimit[0] * numSelected

# find top2
if topLesseeLimit[1] is not None:
    if topLesseeGeq[1]:
        prob += lpSum(var * SortTop(list({lesseeName: value(lpSum(var * lesseeOneHot[lesseeName])) for lesseeName in data['Contract Cust Id'].value_counts().index}.items()), 2)) >= topLesseeLimit[1] * numSelected
    else:
        prob += lpSum(var * SortTop(list({lesseeName: value(lpSum(var * lesseeOneHot[lesseeName])) for lesseeName in data['Contract Cust Id'].value_counts().index}.items()), 2)) <= topLesseeLimit[1] * numSelected


# contract type
for i in range(5):
    if contractLimit[i] is not None:
        if contractGeq[i]:
            prob += lpSum(var * contract[i]) >= contractLimit[i] * numSelected
        else:
            prob += lpSum(var * contract[i]) <= contractLimit[i] * numSelected

# prob.writeLP('problem.lp')

[('MSC', 38593.0)]
[('MSC', 38593.0), ('ONE', 2587.0)]


In [ ]:
solver = PULP_CBC_CMD(msg = True, timeLimit=TIMEOUT, warmStart=True)
prob.solve(solver)

1

In [ ]:
print("==============================================================")
# print(prob)
print("status:", LpStatus[prob.status])
print("==============================================================")
print("target value: ", value(prob.objective))

status: Optimal
target value:  283576587.7500139


In [ ]:
# if solution is found
if debug:
    result = np.array([var[i].varValue for i in range(n)])
    print(int(sum(result)), '/', n, 'containers are selected.')

58750 / 100000 containers are selected.


### Analysis

For debug only

TODO: if infeasible, it may take time much longer than expected

In [ ]:
if debug:    
    print('======================================================================')
    print("nbv: {0} between {1} - {2}".format(round(sum(result * nbv), 2), minTotalNbv, maxTotalNbv))
    print("cost: {0} between {1} - {2}".format(round(sum(result * cost), 2), minTotalCost, maxTotalCost))
    print('billing status: {0}, -- {1}'.format(sum(result * status)/sum(result), OnHireLimit))

    print("container age: ")
    print('\t container average age is {0}, -- {1}'.format(round(sum(result * fleetAgeAvg)/sum(result), 2), fleetAgeAvgLimit))
    for i in range(5):
        if fleetAgeLimit[i] is not None:
            print("\t container age from {0} to {1} is {2}, -- {3}:".format(fleetAgeLowBound[i], fleetAgeUpBound[i], round(sum(result * fleetAge[i])/sum(result), 2), fleetAgeLimit[i]))

    print("weighted age: ")
    print('\t weighted average age is {0}, -- {1}'.format(round(sum(result * weightedAgeAvg)/sum(result), 2), weightedAgeAvgLimit))
    for i in range(5):
        if weightedAgeLimit[i] is not None:
            print("\t weighted age from {0} to {1} is {2}, -- {3}:".format(weightedAgeLowBound[i], weightedAgeUpBound[i], round(sum(result * weightedAge[i])/sum(result), 2), weightedAgeLimit[i]))    

    print("product: ")
    for i in range(5):
        if productLimit[i] is not None:
            print("\t product {0} is {1}, -- {2}:".format(productType[i], round(sum(result * product[i])/sum(result), 2), productLimit[i]))    
   
    
    print("lessee: ")
    for i in range(5):
        if lesseeLimit[i] is not None:
            print("\t lessee {0} is {1}, -- {2}:".format(lesseeType[i], round(sum(result * lessee[i])/sum(result), 2), lesseeLimit[i]))    

    print('Top lessee:')
    numLessee = {lesseeName: value(lpSum(var * lesseeOneHot[lesseeName])) for lesseeName in data['Contract Cust Id'].value_counts().index}
    sortedLessee = list(numLessee.items())
    top3Lessee = heapq.nlargest(3, sortedLessee, key=lambda x:x[1])
    print('\t top 1 {0} is {1}, -- {2}'.format(top3Lessee[0][0], top3Lessee[0][1] / sum(result), topLesseeLimit[0]))
    print('\t top 2 {0} {1} is {2}, -- {3}'.format(top3Lessee[0][0], top3Lessee[1][0], (top3Lessee[0][1] + top3Lessee[1][1])/ sum(result), topLesseeLimit[1]))

    
    print("contract type: ")
    for i in range(5):
        if contractLimit[i] is not None:
            print("\t contract type {0} is {1}, -- {2}:".format(contractType[i], round(sum(result * contract[i])/sum(result), 2), contractLimit[i])) 


nbv: 283576587.75 between 10000000 - 350000000
cost: 299999993.0 between 1000000 - 300000000
billing status: 0.8547744680851064, -- 0.7
container age: 
	 container average age is 1.21, -- 5.0
	 container age from None to 3 is 0.99, -- 0.2:
	 container age from 3 to 6 is 0.0, -- 0.0:
weighted age: 
	 weighted average age is 2.75, -- 7.0
	 weighted age from 3 to 6 is 0.0, -- 0.0:
	 weighted age from 4 to 15 is 0.05, -- 0.01:
product: 
	 product D4H is 0.75, -- 0.5:
	 product D20 is 0.18, -- 0.1:
	 product D40 is 0.0, -- 0.0:
lessee: 
	 lessee MSC is 0.65, -- 0.7:
	 lessee ESSC is 0.01, -- 0.2:
	 lessee ONE is 0.05, -- 0.8:
Top lessee:
	 top 1 MSC is 0.6497872340425532, -- 0.65
	 top 2 MSC ONE is 0.6959659574468086, -- 0.8
contract type: 
	 contract type LT is 0.84, -- 0.6:
	 contract type LP is 0.02, -- 0.02:
	 contract type LM is 0.0, -- 0.0:


### Output

In [ ]:
if prob.status == 1 or prob.status == 2:
    data.drop(['FleetAge1', 'FleetAge2', 'FleetAge3', 'Status', 'WeightedAge1', 'WeightedAge2', 'WeightedAge3', 'ProductType1', 'ProductType2', 'ProductType3', 'Lessee1', 'Lessee2', 'Lessee3', 'ContractType1', 'ContractType2', 'ContractType3'], axis=1, inplace=True)
    data.insert(loc=0, column="selected", value=result)
    data.to_csv('demo_result.csv')

In [ ]:
if prob.status == 1 and debug:
    data.sample(3)